# Exercise 3: Reading and Writing NetCDF files with Python

## Aim: Introduce the netCDF4 library in Python to read and create NetCDF4 files.

### Issues covered:

- Importing netCDF4
- Reading a NetCDF file as a Dataset instance
- Accessing Dimensions, Variables, and Attributes
- Defining Dimensions, Variables, and Attributes
- Writing a NetCDF file as a Dataset

## Let's work with the netCDF4 library to examine the contents of a data file.

Import the `netCDF4` library

In [1]:
from netCDF4 import Dataset


Open the file "../example_data/ggas2014121200_00-18.nc" with the netCDF4 `Dataset` method,
assign it to the `ds` variable.

In [2]:
ds = Dataset('../example_data/ggas2014121200_00-18.nc')

Loop through and print Dataset `variables` names, this is the ID that act like the key to access the Variable.

In [3]:
print(list(ds.variables.keys()))

['longitude', 'latitude', 'surface', 'time', 'CI', 'SSTK', 'MSL', 'TCC', 'U10', 'V10', 'SKT']


From the Dataset `variables`, assign the key `SSTK` to the `sst` variable.
Access the `SSTK` variable like you would a dictionary from `ds.variables`.

- Print the `shape` and `size` of the `sst` variable
- Loop through the `dimensions` of `sst` and print the dimension name and length.
- Loop through the NetCDF attributes of `sst` and print the name and value.
(use `sst.ncattrs()` to access the attributes and `getattr(sst, {attribute name})` to get the value)

In [4]:
sst = ds.variables['SSTK']
print(sst.shape , sst.size )
sst
att_list = sst.ncattrs()
#print(att_list)
for i in att_list:
    v = getattr(sst, i)
    #print(i,':' , v)


(4, 1, 256, 512) 524288


In [5]:
sst

<class 'netCDF4._netCDF4.Variable'>
float32 SSTK(time, surface, latitude, longitude)
    long_name: Sea surface temperature
    units: K
    grid_type: gaussian
    _FillValue: 2e+20
    source: GRIB data
    name: SSTK
    title: Sea surface temperature
    date: 12/12/14
    time: 00:00
unlimited dimensions: time
current shape = (4, 1, 256, 512)
filling on

## Let's extract some data and its related coordinate information and metadata.

Using the `sst` variable from before, take a slice of `sst` as follows and assign it the variable `arr`:

```python
sst[1, 0, 10:20, 30:35]
```

- Print what type of object `arr` is

In [6]:
arr = sst[1, 0, 10:20, 30:35]
arr


masked_array(
  data=[[271.459716796875, 271.459716796875, 271.459716796875,
         271.459716796875, 271.459716796875],
        [271.459716796875, 271.459716796875, 271.459716796875,
         271.459716796875, 271.459716796875],
        [271.498046875, 271.48876953125, 271.4794921875, 271.47021484375,
         271.4609375],
        [270.8381042480469, 270.7830505371094, 270.9228515625,
         271.0626525878906, 271.20245361328125],
        [--, --, --, --, 271.459716796875],
        [271.459716796875, 271.459716796875, 271.459716796875,
         271.459716796875, 271.459716796875],
        [271.459716796875, 271.459716796875, 271.459716796875,
         271.459716796875, 271.459716796875],
        [271.459716796875, 271.459716796875, 271.459716796875,
         271.459716796875, 271.459716796875],
        [272.540771484375, 272.000244140625, 271.459716796875,
         271.459716796875, 271.459716796875],
        [274.676025390625, 274.25634765625, 273.836669921875,
         273.1882

Assign and print the list of `dimensions` from `sst` to the variable `dims`.
Assign the `variables` dictionary of the Dataset, `ds`, to the variables `vars`

In [7]:
#sst.dimensions
vars = ds.variables
type(vars)
vars.keys()

dict_keys(['longitude', 'latitude', 'surface', 'time', 'CI', 'SSTK', 'MSL', 'TCC', 'U10', 'V10', 'SKT'])

In [8]:
dims = sst.dimensions
dims
vars = ds.variables


Now extract the slices from each Dataset variable in `vars` matching those in `arr`
(i.e. matching the values in slice `[1, 0, 10:20, 30:35]` to the values in list `dims`).

Assign them to the following variables:

- Assign `time` slice to `arr_time`
- Assign `surface` slice to `arr_level`
- Assign `latitude` slice to `arr_lats`
- Assign `longitude` slice to `arr_lons`

Print the four new variables.

In [9]:
arr_time = vars['time'][1]
arr_level = vars['surface'][0]
arr_lats = vars['latitude'][10:20]
arr_lons = vars['longitude'][30:35]

Create an empty dictionary called `metadata`, Loop through the NetCDF Variable attributes of `sst` and copy them into this new dictionary.
Use the method as before to get name and value from `sst` and assign them to the key and value of the dictionary.

Print the dictionary.

In [10]:
metadata = {}

#the for loop goes through the attributes in sst by finding them in the list sst.ncaattrs() 
#the second line, for each attribute it adds to the dictionary using the name of the attribute (in the [] ) and assign it to value of that arrribute 
# which is retrieved using getattr(sst, att)
for att in sst.ncattrs():
    metadata[att] = getattr(sst, att)

print(metadata)

{'long_name': 'Sea surface temperature', 'units': 'K', 'grid_type': 'gaussian', '_FillValue': 2e+20, 'source': 'GRIB data', 'name': 'SSTK', 'title': 'Sea surface temperature', 'date': '12/12/14', 'time': '00:00'}


## Let's write the data/metadata from the previous section to a new NetCDF file

In this section, we will define our own Variables, Dimensions, Attributes and save them as a NetCDF file.

To start, import numpy as np.

In [11]:
import numpy as np

Open a new netCDF `Dataset` for writing to a file: "mydata.nc", specify the `mode` as write with "w" and the `format` as "NETCDF4_CLASSIC"
Assign this to the variable `myds`

In [12]:
from netCDF4 import Dataset
from numpy.random import uniform
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

myds = Dataset('mydata.nc', 'w', format='NETCDF4_CLASSIC')

Create four new Dimensions to `myds` from your previous slices. Re-use the names from the last section.
Note that the "level" and "time" Dimensions should have length, "lat" length 10 and "lon" length 5.
To create a new Dimension use `myds.createDimension(name, size)`

In [13]:
time = myds.createDimension('time', 1)
level = myds.createDimension('level', 1)
lat = myds.createDimension('lat', 10)
lon = myds.createDimension('lon', 5)

Create four Variables from those dimensions and assign them following this example for times:

```python
times = myds.createVariable('times', np.float64, ('time',))
```

In [14]:
times = myds.createVariable('times', np.float64, ('time',))
levels = myds.createVariable('levels', np.int32, ('level',))
lats = myds.createVariable('latitudes', np.float32, ('lat',))
lons = myds.createVariable('longitudes', np.float32, ('lon',))

Create `myvar` as a new Dataset Variable, with id "temp", type "np.float64", and dimensions: "time", "level", "lat", "lon".

In [15]:
myvar = myds.createVariable('temp', np.float64, ('time','level','lat','lon'))


Remove the `_FillValue` value in the `metadata` dictionary.
The next step will not work unless we do this.
Fill values should be handled when the Variable is created, but we are ignoring that for this example.

Use `del` to remove the `_FillValue` from `metadata`

In [16]:
del metadata['_FillValue']

Write Variable attributes from the key/value pairs found in the input data (held in the `metadata` dictionary) to `myvar`.
Use `myvar.setncattr(key, value)` to write attributes to the Dataset.

Write one more global attribute "source" to `myds`. Set the attribute `source` to the value "super dataset".
Use `myds.source` to create and set the value to the global attribute.

In [17]:
for key, value in metadata.items():
    myvar.setncattr(key, value)
print(myvar)
myds.source = "super dataset"

<class 'netCDF4._netCDF4.Variable'>
float64 temp(time, level, lat, lon)
    long_name: Sea surface temperature
    units: K
    grid_type: gaussian
    source: GRIB data
    name: SSTK
    title: Sea surface temperature
    date: 12/12/14
    time: 00:00
unlimited dimensions: 
current shape = (1, 1, 10, 5)
filling on, default _FillValue of 9.969209968386869e+36 used


Assign the `arr` array from before to `myvar[:]`

Write some data values to each of your four spatio-temporal variables you created.
Use simple lists of integers for these.
Make sure they are the right length matching the slices from the previous section,
use the index `[:]` on your `myds` Variables and assign the created array variables to them.

Lastly close the Dataset, `myds`, to save the file.

In [18]:
myvar[:] = arr
times [:] = [1]
levels[:] = [1]
lats [:] = np.arange(0, 40, 4)
lons [:] = np.arange(0, 20, 4)


np.shape(myvar)

myds.close()

To view the file contents, you can use the Jasmin service to run the `$ ncdump mydata.nc` command from the directory where the file is saved. Alternatively, you can call out to the linux shell from a Notebook, using: 

```
!ncdump mydata.nc
```

In [19]:
!ncdump mydata.nc

netcdf mydata {
dimensions:
	time = 1 ;
	level = 1 ;
	lat = 10 ;
	lon = 5 ;
variables:
	double times(time) ;
	int levels(level) ;
	float latitudes(lat) ;
	float longitudes(lon) ;
	double temp(time, level, lat, lon) ;
		temp:long_name = "Sea surface temperature" ;
		temp:units = "K" ;
		temp:grid_type = "gaussian" ;
		temp:source = "GRIB data" ;
		temp:name = "SSTK" ;
		temp:title = "Sea surface temperature" ;
		temp:date = "12/12/14" ;
		temp:time = "00:00" ;

// global attributes:
		:source = "super dataset" ;
data:

 times = 1 ;

 levels = 1 ;

 latitudes = 0, 4, 8, 12, 16, 20, 24, 28, 32, 36 ;

 longitudes = 0, 4, 8, 12, 16 ;

 temp =
  271.459716796875, 271.459716796875, 271.459716796875, 271.459716796875, 
    271.459716796875,
  271.459716796875, 271.459716796875, 271.459716796875, 271.459716796875, 
    271.459716796875,
  271.498046875, 271.48876953125, 271.4794921875, 271.47021484375, 271.4609375,
  270.838104248047, 270.783050537109, 270.9228515625, 271.062652587891, 
    271

In [20]:
!ncdump -h mydata.nc

netcdf mydata {
dimensions:
	time = 1 ;
	level = 1 ;
	lat = 10 ;
	lon = 5 ;
variables:
	double times(time) ;
	int levels(level) ;
	float latitudes(lat) ;
	float longitudes(lon) ;
	double temp(time, level, lat, lon) ;
		temp:long_name = "Sea surface temperature" ;
		temp:units = "K" ;
		temp:grid_type = "gaussian" ;
		temp:source = "GRIB data" ;
		temp:name = "SSTK" ;
		temp:title = "Sea surface temperature" ;
		temp:date = "12/12/14" ;
		temp:time = "00:00" ;

// global attributes:
		:source = "super dataset" ;
}
